# Set up

In [1]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [2]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [3]:
import time
import pandas as pd
from sqlalchemy import create_engine
from trino.dbapi import connect
from datetime import datetime, timedelta

# Schemas 
Here we created 3 schema:  
1. presence_data: mainly for test purpose 
2. pop_density: keep aggregated population density data from device location 
3. od_matrix: aggregated OD matrix from trajectory dataset, only cell has more than 10 unique user from dl have been selected here.

In [4]:
%%sql
show schemas from dedicated

 * trino://localhost:9090/cuebiq/
Done.


,Schema
0,censustocensus
1,dataflowtest
2,default
3,ds_data_for_complexity
4,ds_data_for_complexity_v2
5,information_schema
6,od_matrix
7,od_matrix20
8,od_matrix_10
9,pop_density


In [8]:
# from sqlalchemy.engine import create_engine
# output_schema_name = 'presence_data'
# # output_schema_name = 'pop_density'
# # output_schema_name = 'od_matrix'
# # output_schema_name = "20OD_matrix"

# con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

In [4]:
%%sql
show tables from dedicated.pop_density

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,pd_co_20191004_agg3
1,pd_co_20191004_agg5
2,pd_co_20191005_agg3
3,pd_co_20191005_agg5
4,pd_co_20191006_agg3
...,...
1632,pd_mx_20200527_agg3
1633,pd_mx_20200528_agg3
1634,pd_mx_20200529_agg3
1635,pd_mx_20200530_agg3


In [5]:
%%sql
show tables from dedicated.od_matrix_10

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,od_co_20191101_agg3_10
1,od_co_20191102_agg3_10
2,od_co_20191103_agg3_10
3,od_co_20191104_agg3_10
4,od_co_20191105_agg3_10
...,...
2249,od_mx_20191230_agg3_10
2250,od_mx_20191230_agg5_10
2251,od_mx_20191231_agg3_10
2252,od_mx_20191231_agg5_10


In [8]:
%%sql
show tables from dedicated.presence_data

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,od_id_20190101_agg3
1,od_id_20190101_agg5
2,od_id_20190102_agg3
3,od_id_20190102_agg5
4,od_id_20190103_agg3
...,...
445,pd_id_2019_agg3
446,pd_id_2019_agg5
447,pd_in_20190101_agg3
448,pd_in_20190102_agg3


In [38]:
%%sql
show tables from dedicated.od_matrix_10 LIKE 'od_mx_2019%_agg5_10'

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,od_mx_20190101_agg5_10
1,od_mx_20190102_agg5_10
2,od_mx_20190103_agg5_10
3,od_mx_20190104_agg5_10
4,od_mx_20190105_agg5_10
...,...
360,od_mx_20191227_agg5_10
361,od_mx_20191228_agg5_10
362,od_mx_20191229_agg5_10
363,od_mx_20191230_agg5_10


## Create schema

In [11]:
output_schema_name = 'OD_matrix_10'

In [12]:
%sql create schema if not exists dedicated.$output_schema_name

 * trino://localhost:9090/cuebiq/
Done.


""


## Create table 

In [ ]:
# output_schema_name = 'presence_data'
# table_name = f"dedicated.{output_schema_name}.new_table_demo"

In [ ]:
# %sql create table if not exists $table_name (col_0 varchar, col_1 bigint, col_2 varchar)

In [10]:
# %sql show tables from dedicated.presence_data

## Delete table

In [5]:
# %%sql
# DROP TABLE IF EXISTS dedicated.od_matrix_10.od_co_20191115_agg5_10

# # pop_density presence_data od_matrix

 * trino://localhost:9090/cuebiq/


NameError: name 'DatabaseError' is not defined

## Rename table

In [89]:
# Rename the table to the final name based on date and country code

output_schema_name = 'od_matrix'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

rename_table_query = f"""
ALTER TABLE "od_id_20190301_test_agg5" RENAME TO "od_id_20190301_agg5"
"""

# Execute the rename table query
with con.connect() as connection:
    connection.execute(rename_table_query)

OperationalError: (trino.exceptions.TrinoExternalError) TrinoExternalError(type=EXTERNAL, name=HIVE_METASTORE_ERROR, message="New location for this table hive.od_matrix.od_id_20190301_agg5 already exists : s3a://cuebiq-paas-org-1872-kr7pteibwb/metastore/od_matrix.db/od_id_20190301_agg5", query_id=20240606_145713_00482_44guw)
[SQL: 
ALTER TABLE "od_id_20190301_test_agg5" RENAME TO "od_id_20190301_agg5"
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

## Check table

In [4]:
# check
# q2 = f"select * from dedicated.presence_data.od_id_20190103_agg3"
q2 = f"select * from dedicated.od_matrix_10.od_mx_2019_agg5"

%sql $q2

 * trino://localhost:9090/cuebiq/
Done.


,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_date
0,9sw3q,9sw3w,13,68.715385,27.966667,85.097398,5982.104104,4456.015808,5984.604327,60.230769,5.0,140.118494,20190106
1,9g9f2,9g9f0,39,60.115812,30.383333,89.130775,7875.425172,6202.787986,8641.216625,6.410256,6.0,3.400603,20190106
2,9g9f3,9g9f0,10,51.630000,42.641667,24.452616,9756.155180,9157.371230,2555.009380,6.800000,6.0,3.552777,20190106
3,9gd9n,9gd9n,21,54.924603,37.816667,45.782087,2196.370448,844.283352,3446.413389,4.809524,5.0,2.315579,20190106
4,9g6sf,9g6sf,17,103.891176,41.733333,174.601325,7118.566541,1333.845203,16562.857931,5.588235,5.0,2.980328,20190106
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087832,9uddv,9udej,30,71.956111,50.033333,72.624866,5602.742897,5105.047627,4079.921585,7.566667,7.0,4.804332,20191003
2087833,9v0ny,9v0nv,17,71.298039,28.750000,115.036237,4257.838845,4190.294103,2142.030550,6.117647,6.0,2.642971,20191003
2087834,d58r9,d58rb,10,51.691667,43.708333,35.769733,26929.889970,7994.983344,59174.695210,7.700000,5.0,5.963780,20191003
2087835,9mvcj,9mvcj,10,29.998333,21.050000,26.969554,2570.812973,269.686125,7285.404499,4.200000,3.0,3.011091,20191003


In [8]:
# check
q2 = f"select DISTINCT * from dedicated.od_matrix_10.od_in_2019_agg5"
%sql $q2

 * trino://localhost:9090/cuebiq/
Done.


,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_date
0,tuu07,tuu07,20,49.024167,36.500000,39.283400,16450.004255,1083.217784,31947.542019,5.200000,5.0,1.673320,20190224
1,temhu,temhu,24,101.402778,73.600000,104.831827,4544.751940,1690.679423,8407.110935,6.500000,5.0,4.951943,20190224
2,tsvch,tsvck,19,65.105263,28.266667,95.368958,7066.974928,4734.496720,13119.923534,6.000000,4.0,4.509250,20190224
3,ts5ej,ts5ej,341,70.552981,31.833333,97.924019,3235.170007,915.182642,8814.456579,5.302053,4.0,4.331787,20190224
4,tsjb8,tsjb8,12,127.104167,59.933333,228.328938,631.995203,329.680113,944.259713,3.666667,2.5,2.461830,20190224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388255,wh9k9,wh9kb,10,130.755000,64.900000,206.327867,10325.905650,6560.114250,8156.976724,8.600000,6.0,7.486284,20191026
1388256,tek4g,tek4g,21,27.357937,20.916667,24.815150,735.286348,272.742098,911.804136,3.238095,3.0,1.261141,20191026
1388257,tst06,tst06,10,49.028333,39.416667,47.145936,2488.341912,968.206289,4068.393096,3.700000,2.0,2.626785,20191026
1388258,ttqzf,ttqzf,86,60.622287,34.475000,102.418037,1621.489067,906.900418,1797.105312,5.941860,5.0,3.137457,20191026


In [30]:
# check
q1 = f"select * from dedicated.pop_density20.pd_co_2020_agg3"
%sql $q1

 * trino://localhost:9090/cuebiq/
Done.


,geohash_3,no_of_points,no_of_unique_users,local_date
0,6rf,4356,2600,20200116
1,6rg,9327,5497,20200116
2,6ru,2343,1520,20200116
3,6rv,3292,2070,20200116
4,6ry,95,67,20200116
...,...,...,...,...
17122,d3m,392,19,20201109
17123,d3u,809,33,20201109
17124,d4j,1108,33,20201109
17125,d65,492,27,20201109


Check duplicates

In [7]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [8]:
# Construct the SQL query
query = f"""
SELECT DISTINCT *
FROM dedicated.od_matrix_10.od_mx_2019_agg5
"""
# presence_data, od_matrix_10

# query = """
# SELECT DISTINCT
#     start_geohash5,
#     end_geohash5,
#     trip_count,
#     ROUND(m_duration_min, 6) AS rounded_m_duration_min,
#     ROUND(mdn_duration_min, 6) AS rounded_mdn_duration_min,
#     ROUND(sd_duration_min, 6) AS rounded_sd_duration_min,
#     ROUND(m_length_m, 6) AS rounded_m_length_m,
#     ROUND(mdn_length_m, 6) AS rounded_mdn_length_m,
#     ROUND(sd_length_m, 6) AS rounded_sd_length_m,
#     ROUND(m_points_no, 6) AS rounded_m_points_no,
#     ROUND(mdn_points_no, 6) AS rounded_mdn_points_no,
#     ROUND(sd_points_no, 6) AS rounded_sd_points_no
# FROM
#     dedicated.od_matrix_10.od_in_20190428_agg5_10
# """

# od_matrix pop_density
pe_dl_table_gen = sql_engine.read_sql(query)
pe_dl_table_gen

,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_date
0,9ghwh,9ghwh,26,48.689103,35.650000,36.456864,7162.207133,1281.356851,20349.526498,6.038462,5.0,3.852072,20190207
1,9s1fd,9s1fd,11,87.715152,56.500000,101.770637,4575.926453,3714.720759,3769.837724,5.727273,5.0,2.412091,20190207
2,9t1jf,9t1jc,26,39.501282,28.266667,33.620355,4930.394756,3874.753205,4841.629644,5.423077,5.0,2.685860,20190207
3,9thdf,9the1,88,73.804356,44.083333,97.824702,6568.517100,5588.075286,4735.397013,7.079545,6.0,4.480408,20190207
4,9u8dj,9u8cb,12,149.423611,78.733333,150.591103,16301.335282,15713.568565,3266.840356,14.250000,11.0,13.531949,20190207
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087832,9u8y8,9u8yb,22,92.021212,50.350000,150.130597,4460.007708,2985.316412,3823.882357,14.318182,6.0,23.743443,20191225
2087833,9udej,9udeh,48,65.229167,32.233333,86.014723,6559.801429,3985.674614,11476.372130,7.354167,4.0,7.864517,20191225
2087834,9g3mw,9g3my,32,90.920313,50.808333,99.204252,17728.899819,4515.233086,58738.876226,7.500000,5.5,5.962247,20191225
2087835,9g3my,9g3w3,11,80.587879,55.816667,43.798507,28213.744594,26258.808063,10065.189265,18.909091,12.0,11.536503,20191225


In [9]:
# pe_dl_table_gen1 = pe_dl_table_gen.drop(columns=['partition_key'])
pe_dl_table_gen1 = pe_dl_table_gen
pe_dl_table_gen1

,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_date
0,9ghwh,9ghwh,26,48.689103,35.650000,36.456864,7162.207133,1281.356851,20349.526498,6.038462,5.0,3.852072,20190207
1,9s1fd,9s1fd,11,87.715152,56.500000,101.770637,4575.926453,3714.720759,3769.837724,5.727273,5.0,2.412091,20190207
2,9t1jf,9t1jc,26,39.501282,28.266667,33.620355,4930.394756,3874.753205,4841.629644,5.423077,5.0,2.685860,20190207
3,9thdf,9the1,88,73.804356,44.083333,97.824702,6568.517100,5588.075286,4735.397013,7.079545,6.0,4.480408,20190207
4,9u8dj,9u8cb,12,149.423611,78.733333,150.591103,16301.335282,15713.568565,3266.840356,14.250000,11.0,13.531949,20190207
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087832,9u8y8,9u8yb,22,92.021212,50.350000,150.130597,4460.007708,2985.316412,3823.882357,14.318182,6.0,23.743443,20191225
2087833,9udej,9udeh,48,65.229167,32.233333,86.014723,6559.801429,3985.674614,11476.372130,7.354167,4.0,7.864517,20191225
2087834,9g3mw,9g3my,32,90.920313,50.808333,99.204252,17728.899819,4515.233086,58738.876226,7.500000,5.5,5.962247,20191225
2087835,9g3my,9g3w3,11,80.587879,55.816667,43.798507,28213.744594,26258.808063,10065.189265,18.909091,12.0,11.536503,20191225


In [10]:
pe_dl_table_gen1.columns

Index(['start_geohash5', 'end_geohash5', 'trip_count', 'm_duration_min',
       'mdn_duration_min', 'sd_duration_min', 'm_length_m', 'mdn_length_m',
       'sd_length_m', 'm_points_no', 'mdn_points_no', 'sd_points_no',
       'local_date'],
      dtype='object')

In [11]:
df_no_duplicates = pe_dl_table_gen1.drop_duplicates()
# df_no_duplicates.sort_values('no_of_unique_users')
len(df_no_duplicates)

2087837

In [12]:
df_no_duplicates
df_no_duplicates.to_csv('/home/jovyan/Data/TJ/od_mx_2019_agg5_1.csv', index=False)

In [50]:
# Test
# Filter rows where start_geohash5 and end_geohash5 are both 'tsvcq'
filtered_rows = pe_dl_table_gen[(pe_dl_table_gen['start_geohash5'] == 'qqy81') & (pe_dl_table_gen['end_geohash5'] == 'qqy81')]
filtered_rows

,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_date
0,qqy81,qqy81,71,89.959390,19.466667,183.734745,6557.431564,1036.287394,27089.014225,5.985915,4.0,8.845488,20190125
2622,qqy81,qqy81,183,68.749727,29.650000,114.805019,2658.631866,972.758340,5324.731976,5.409836,4.0,7.993219,20190402
4420,qqy81,qqy81,214,68.772118,29.208333,104.622856,3558.882879,996.034931,6047.838214,5.275701,4.0,4.528850,20190314
7333,qqy81,qqy81,102,45.888072,17.683333,69.878520,3059.027512,880.188031,8879.545712,5.470588,4.0,7.907517,20190201
9798,qqy81,qqy81,60,81.334167,21.000000,155.973026,3998.844919,1363.548415,6442.115208,5.716667,4.0,6.109431,20190107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
826777,qqy81,qqy81,185,38.601441,16.533333,68.911125,2427.131500,1126.265626,3802.659851,5.145946,4.0,3.882135,20190729
831913,qqy81,qqy81,233,62.816381,25.750000,118.465804,6160.543154,918.782546,23969.452585,5.798283,5.0,5.656671,20191116
832810,qqy81,qqy81,430,52.059690,17.016667,103.456542,3173.608552,793.056870,9865.801451,4.906977,4.0,3.570887,20191017
835310,qqy81,qqy81,311,67.469185,21.900000,141.562766,3351.146068,725.578560,13362.470260,4.784566,4.0,3.601329,20190912


remove duplicates for OD

In [39]:
# Round floating-point columns to a consistent number of decimal places
float_cols = ['m_duration_min', 'mdn_duration_min', 'sd_duration_min', 'm_length_m', 'mdn_length_m', 'sd_length_m', 'm_points_no', 'mdn_points_no', 'sd_points_no']
pe_dl_table_gen[float_cols] = pe_dl_table_gen[float_cols].round(6)
pe_dl_table_gen

,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no
0,d2g68,d2g4x,10,44.331667,34.691667,27.473638,5573.072206,5112.658480,3233.439158,8.400000,7.5,4.550946
1,d3442,d3442,22,83.034091,17.850000,142.368566,1869.122673,444.950846,4946.839905,4.954545,3.5,4.369562
2,d34k7,d34k7,62,76.598387,20.441667,150.985507,1457.202199,377.282107,6184.804976,4.500000,3.5,3.762411
3,d3fy9,d3fy3,186,82.076882,38.266667,106.198712,5683.388992,4050.209845,6086.877307,8.053763,5.0,8.755990
4,d2gtt,d2gtt,23,83.470290,65.933333,76.275841,1548.594681,1248.833667,1357.090396,5.695652,5.0,3.698093
...,...,...,...,...,...,...,...,...,...,...,...,...
9697,d3hp9,d3hpc,20,102.007500,31.391667,224.251630,1938.158800,1536.592762,1201.985630,4.000000,4.0,1.414214
9698,d3hpc,d3hpc,99,78.702694,22.500000,135.161447,1568.576179,596.254687,3175.734271,4.616162,4.0,2.605751
9699,d347b,d34k0,75,115.550000,47.850000,178.847136,5497.843858,3934.303028,5884.639466,6.533333,5.0,4.674061
9700,d3478,d3472,311,88.693676,51.300000,113.812173,5844.935528,4673.087340,6795.179322,8.061093,6.0,6.712492


In [40]:
df_deduplicated = pe_dl_table_gen.drop_duplicates()
df_deduplicated

,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no
0,d2g68,d2g4x,10,44.331667,34.691667,27.473638,5573.072206,5112.658480,3233.439158,8.400000,7.5,4.550946
1,d3442,d3442,22,83.034091,17.850000,142.368566,1869.122673,444.950846,4946.839905,4.954545,3.5,4.369562
2,d34k7,d34k7,62,76.598387,20.441667,150.985507,1457.202199,377.282107,6184.804976,4.500000,3.5,3.762411
3,d3fy9,d3fy3,186,82.076882,38.266667,106.198712,5683.388992,4050.209845,6086.877307,8.053763,5.0,8.755990
4,d2gtt,d2gtt,23,83.470290,65.933333,76.275841,1548.594681,1248.833667,1357.090396,5.695652,5.0,3.698093
...,...,...,...,...,...,...,...,...,...,...,...,...
8491,d2g68,d2g6c,16,170.830208,87.950000,276.991917,10166.490605,9145.258092,5242.948725,11.937500,10.0,9.161286
8720,d24jb,d24jb,14,49.328571,33.550000,50.966920,394.397625,328.097523,315.826857,3.714286,3.5,1.382783
9222,d4jzs,d4jzt,26,86.152564,56.841667,131.632234,2103.749848,1044.795943,2642.664591,6.692308,5.0,5.401994
9303,d2fk9,d2fkc,29,88.008621,57.683333,82.996779,7791.678004,4598.609769,9972.860488,11.379310,7.0,16.817112


In [24]:
pe_dl_table_gen['start_geohash5'].unique()

array(['qqxem', 'qw6pb', 'qqy81', 'w21ym', 'qqgup', 'qqu88', 'qquh1',
       'qqgmt', 'w204g', 'qqw7n', 'qw851', 'qxm46', 'qqts7', 'qqv1t',
       'qw8ny', 'qw8p6', 'w0umy', 'w21vv', 'qqguj', 'qqgun', 'qpz6g',
       'qqvbz', 'qqgu7', 'qqguq', 'qquj3', 'qqguh', 'qquhg', 'qw90m',
       'qx3sk', 'w204u', 'qqgud', 'qqguw', 'qqv1w', 'qqv1y', 'qqggv',
       'qquh0', 'qqwe2', 'qqggw', 'qqwef', 'qquk4', 'qqgux', 'qqxjz',
       'qw819', 'qw936', 'qxg0f', 'qqww4', 'w21vz', 'w21yr', 'qquh4',
       'qqxes', 'qqyc8', 'w24m4', 'qpzwd', 'qquhb', 'qqxj0', 'qxhfy',
       'rp1q4', 'qqgfm', 'qquhj', 'qqw7r', 'qqxx6', 'qw3yw', 'w204e',
       'qw3vu', 'qw6x4', 'qw3yz', 'w0whj', 'qqgum', 'qquhs', 'qqwe8',
       'qqwvy', 'qr9wy', 'qw3zw', 'qxhfs', 'qzv6u', 'qpz6e', 'qqu89',
       'qqu8c', 'qw8c7', 'w0wdt', 'qqww6', 'qxg0d', 'qqttt', 'qqu82',
       'qw3vg', 'qw8nn', 'qqz8v', 'qw3yv', 'qw6x8', 'qxhft', 'qqgfy',
       'qw84d', 'qw84e', 'qw8jv', 'qqw76', 'qqu09', 'qqu81', 'w0whv',
       'qqz08', 'qw9

In [10]:
pe_dl_table_gen['local_date'].nunique()

61

In [47]:
unique_values_count = pe_dl_table_gen["local_date"].unique()
# unique_values_count = pe_dl_table_gen["local_date"].nunique()
unique_values_count.sort()
unique_values_count

array(['20190101', '20190102', '20190103', '20190104', '20190105',
       '20190106', '20190107', '20190108', '20190109', '20190110',
       '20190111', '20190112', '20190113', '20190114', '20190115',
       '20190116', '20190117', '20190118', '20190119', '20190120',
       '20190121', '20190122', '20190123', '20190124', '20190125',
       '20190126', '20190127', '20190128', '20190129', '20190130',
       '20190131', '20190201', '20190202', '20190203', '20190204',
       '20190205', '20190206', '20190207', '20190208', '20190209',
       '20190210', '20190211', '20190212', '20190213', '20190214',
       '20190215', '20190216', '20190217', '20190218', '20190219',
       '20190220', '20190221', '20190222', '20190223', '20190224',
       '20190225', '20190226', '20190227', '20190228', '20190301',
       '20190302', '20190303', '20190304', '20190305', '20190306',
       '20190307', '20190308', '20190309', '20190310', '20190311',
       '20190312', '20190313', '20190314', '20190315', '201903

In [20]:
# Generate all dates in 2020
all_dates_2020 = pd.date_range(start='2020-01-01', end='2020-12-31')

# Find the missing dates
missing_dates = all_dates_2020.difference(unique_values_count)

# Display the missing dates
missing_dates = missing_dates.strftime('%Y%m%d')
missing_dates

Index(['20200104', '20200130', '20200429', '20200612'], dtype='object')